In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#3.2
#Ebenso habe ich auch noch das von google entwickelte Plattform Tensorflow genommen
#Das "Sequential" im Tensorflow ermöglicht es mir neuronale Netze sequenziell zu stapeln.
#Dense ist eine klasse die eine schicht ein neuronales Netzwerk definiert
#Dropout ist eine Technik zur Regulierung von neuronalen Netzen
#(Der rest folgt dann bei der implementierung)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [46]:
#Ich habe mich entschieden das ich für meine Berechnungen die Spalten 'Date', 'Unnamed: 0','4046','4225','4770','year','region nicht brauche
#Ebenso wollte ich nicht das er mir eine Kopie gibt sondern es soll direkt verändert werden.
df=pd.read_csv("avocado.csv")
df.drop(['Date','number','4046','4225','4770','year','region'], axis=1,inplace=True)
df

,AveragePrice,TotalVolume,TotalBags,SmallBags,LargeBags,XLargeBags,type
0,1.33,64236.62,8696.87,8603.62,93.25,0.0,conventional
1,1.35,54876.98,9505.56,9408.07,97.49,0.0,conventional
2,0.93,118220.22,8145.35,8042.21,103.14,0.0,conventional
3,1.08,78992.15,5811.16,5677.40,133.76,0.0,conventional
4,1.28,51039.60,6183.95,5986.26,197.69,0.0,conventional
...,...,...,...,...,...,...,...
18244,1.63,17074.83,13498.67,13066.82,431.85,0.0,organic
18245,1.71,13888.04,9264.84,8940.04,324.80,0.0,organic
18246,1.87,13766.76,9394.11,9351.80,42.31,0.0,organic
18247,1.93,16205.22,10969.54,10919.54,50.00,0.0,organic


In [47]:
# Hier habe ich noch den 'type' zu einer nummer konvertiert
one_hot_encoder = OneHotEncoder()
type_encoded = one_hot_encoder.fit_transform(df[['type']])
type_encoded = pd.DataFrame(type_encoded.toarray(), columns=['conventional','organic'])
df_encoded = pd.concat([df, type_encoded], axis=1)
df_encoded = df_encoded.drop(['type'], axis=1)

In [48]:
df_encoded

,AveragePrice,TotalVolume,TotalBags,SmallBags,LargeBags,XLargeBags,conventional,organic
0,1.33,64236.62,8696.87,8603.62,93.25,0.0,1.0,0.0
1,1.35,54876.98,9505.56,9408.07,97.49,0.0,1.0,0.0
2,0.93,118220.22,8145.35,8042.21,103.14,0.0,1.0,0.0
3,1.08,78992.15,5811.16,5677.40,133.76,0.0,1.0,0.0
4,1.28,51039.60,6183.95,5986.26,197.69,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
18244,1.63,17074.83,13498.67,13066.82,431.85,0.0,0.0,1.0
18245,1.71,13888.04,9264.84,8940.04,324.80,0.0,0.0,1.0
18246,1.87,13766.76,9394.11,9351.80,42.31,0.0,0.0,1.0
18247,1.93,16205.22,10969.54,10919.54,50.00,0.0,0.0,1.0


In [49]:
#Hier habe ich das Datenset in training und testing eingeteilt
X = df_encoded.drop(['AveragePrice'], axis=1)
y = df_encoded['AveragePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#Hier habe ich die numerischen Merkmale der Trainings- und Testdatensätze skaliert mithilfe von StandartScaler
numeric_transformer = StandardScaler()
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
ct = ColumnTransformer([('numeric', numeric_transformer, numeric_features)], remainder='passthrough')
X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.transform(X_test)

In [50]:
#Neuronale Netzwerk architektur definieren
#Habe habe das Neuronale Netzwerk von Tensorflow genommen, da ich das einfacher finde und mit dem Neuronalen Netzwerk auf sklearn sehr Probleme hatte.
#Es hatte viele gute Turtorials dazu und uch habe gehört es besser ist als das von sklearn.
#Ebenso war es auch sehr einfach zu implementieren und zu konfigurieren.
model = Sequential([
    Dense(128, input_shape=(X_train_scaled.shape[1],), activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')
])

# Kompiliert das Modell und eine Zusammenfassung des Modells
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               1024      
                                                                 
 dropout_8 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 9,345
Trainable params: 9,345
Non-trainable params: 0
_________________________________________________________________


In [51]:
#Das Model Trainieren
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=2)


Epoch 1/50
457/457 - 3s - loss: 0.1979 - mae: 0.3239 - 3s/epoch - 6ms/step
Epoch 2/50
457/457 - 1s - loss: 0.1285 - mae: 0.2744 - 1s/epoch - 3ms/step
Epoch 3/50
457/457 - 1s - loss: 0.1195 - mae: 0.2653 - 1s/epoch - 3ms/step
Epoch 4/50
457/457 - 1s - loss: 0.1130 - mae: 0.2585 - 1s/epoch - 2ms/step
Epoch 5/50
457/457 - 1s - loss: 0.1099 - mae: 0.2549 - 1s/epoch - 2ms/step
Epoch 6/50
457/457 - 1s - loss: 0.1074 - mae: 0.2515 - 1s/epoch - 3ms/step
Epoch 7/50
457/457 - 1s - loss: 0.1044 - mae: 0.2475 - 1s/epoch - 3ms/step
Epoch 8/50
457/457 - 1s - loss: 0.1031 - mae: 0.2458 - 1s/epoch - 2ms/step
Epoch 9/50
457/457 - 1s - loss: 0.1000 - mae: 0.2417 - 1s/epoch - 3ms/step
Epoch 10/50
457/457 - 1s - loss: 0.0992 - mae: 0.2415 - 1s/epoch - 3ms/step
Epoch 11/50
457/457 - 1s - loss: 0.0980 - mae: 0.2396 - 1s/epoch - 2ms/step
Epoch 12/50
457/457 - 1s - loss: 0.0969 - mae: 0.2374 - 1s/epoch - 3ms/step
Epoch 13/50
457/457 - 1s - loss: 0.0953 - mae: 0.2348 - 1s/epoch - 2ms/step
Epoch 14/50
457/457 -

In [52]:
# Das Model mit "X_test_scaled" und "y_test" trainieren
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=50, batch_size=32)


Epoch 1/50
457/457 [==============================] - 2s 4ms/step - loss: 0.0881 - mae: 0.2251 - val_loss: 0.0880 - val_mae: 0.2251
Epoch 2/50
457/457 [==============================] - 2s 3ms/step - loss: 0.0883 - mae: 0.2247 - val_loss: 0.0881 - val_mae: 0.2281
Epoch 3/50
457/457 [==============================] - 2s 3ms/step - loss: 0.0877 - mae: 0.2241 - val_loss: 0.0888 - val_mae: 0.2286
Epoch 4/50
457/457 [==============================] - 1s 3ms/step - loss: 0.0878 - mae: 0.2245 - val_loss: 0.0879 - val_mae: 0.2270
Epoch 5/50
457/457 [==============================] - 2s 3ms/step - loss: 0.0876 - mae: 0.2247 - val_loss: 0.0885 - val_mae: 0.2263
Epoch 6/50
457/457 [==============================] - 2s 4ms/step - loss: 0.0868 - mae: 0.2230 - val_loss: 0.0879 - val_mae: 0.2266
Epoch 7/50
457/457 [==============================] - 2s 4ms/step - loss: 0.0878 - mae: 0.2244 - val_loss: 0.0882 - val_mae: 0.2281
Epoch 8/50
457/457 [==============================] - 2s 4ms/step - loss: 0.

In [53]:
#3.3
#Modell auf dem Testdatensatz evaluiren
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test.to_numpy().reshape(-1), y_pred.reshape(-1))
mae = np.mean(np.abs(y_pred.reshape(-1) - y_test.to_numpy().reshape(-1)))

print(y_pred[:10])
print("MSE:", mse)
print("MAE:", mae)
#Die Ergebnisse zeigen, dass das Modell bei der Vorhersage der Testdaten eine relativ geringe mittlere quadratische Abweichung (MSE) von 0,085 aufweist,
#was darauf hinweist, dass es eine gute Genauigkeit aufweist.
#Ebenso zeigt das mittlere absolute Fehler (MAE) Ergebnis von 0,221, dass die Vorhersage des Modells im Durchschnitt um 0,221 Einheiten vom tatsächlichen Wert abweicht.
#Insgesamt scheint das Modell bei der Vorhersage der Testdaten gut abzuschneiden.

115/115 [==============================] - 0s 2ms/step
[[1.1817518]
 [1.1749743]
 [1.755512 ]
 [1.2138189]
 [1.7375143]
 [1.7296013]
 [1.7983912]
 [1.0633618]
 [1.7339671]
 [1.1941186]]
MSE: 0.08594827758297875
MAE: 0.22445521709690355
